In [ ]:
import gc
import os
import torch
import random
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib
import torchvision
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import matplotlib.patches as patches
from sklearn.utils import resample
import torchvision.models as models
from torch.nn import functional as F
from sklearn.preprocessing import StandardScaler
from torchvision import datasets, models
from sklearn.metrics import accuracy_score, roc_curve, auc, classification_report, confusion_matrix
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import GroupKFold 

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu') 

In [ ]:
# paths to patch directories 

patch_size = {patch_size}

patch_dir_tnbc = 'PATH-TO-IMPRESS-TNBC-PATCH-DIR'
patch_dir_her2 = 'PATH-TO-IMPRESS-HER2-PATCH-DIR'

In [ ]:
# paths to patch embeddings 

her2_feats_path = 'PATH-TO-IMPRESS-TNBC-PATCH-EMBEDDINGS-FILE(.pth)' 
tnbc_feats_path = 'PATH-TO-IMPRESS-HER2-PATCH-EMBEDDINGS-FILE(.pth)'

**LOADING PATCH DATA**

In [ ]:
# reading patch info, and obtaining patch paths and labels 

her2_df = pd.read_csv('PATH-TO-PATCH-INFO-DATA-CSV')
tnbc_df = pd.read_csv('PATH-TO-PATCH-INFO-DATA-CSV')

def obtain_patch_path(row, patch_dir): 
    
    wsi = str(row['WSI No.'])
    x = str(row['X Pixel'])
    y = str(row['Y Pixel'])
    
    if len(wsi) == 2: 
        return f'{patch_dir}/0{wsi}_{x}_{y}.jpg'
    else: 
        return f'{patch_dir}/{wsi}_{x}_{y}.jpg'

her2_df['Patch Path'] = her2_df.apply(lambda row: obtain_patch_path(row, patch_dir_her2), axis=1)
tnbc_df['Patch Path'] = tnbc_df.apply(lambda row: obtain_patch_path(row, patch_dir_tnbc), axis=1)

# patch labels 
her2_df['Label'] = [1] * len(her2_df) 
tnbc_df['Label'] = [0] * len(tnbc_df) 

In [ ]:
tnbc_df = tnbc_df[tnbc_df['Annotation Id'] != -1]
her2_df = her2_df[her2_df['Annotation Id'] != -1]

tnbc_df = tnbc_df.sort_values(by=["WSI No.", "Annotation Id", "Y Pixel", "X Pixel"])
her2_df = her2_df.sort_values(by=["WSI No.", "Annotation Id", "Y Pixel", "X Pixel"])

In [ ]:
df = pd.concat([tnbc_df, her2_df], ignore_index=True) 
df.head(2) 

**LOADING PATCH FEATURES** 

In [ ]:
# loading feature embeddings for patches 

patch_features_temp_her2 = torch.load(her2_feats_path, map_location='cpu', weights_only = False) 
patch_features_temp_tnbc = torch.load(tnbc_feats_path, map_location='cpu', weights_only = False) 

tnbc_indexes = tnbc_df['Embedding No.'].to_list() 
her2_indexes = her2_df['Embedding No.'].to_list() 

patch_features_temp_tnbc = [patch_features_temp_tnbc[x] for x in tnbc_indexes] 
patch_features_temp_her2 = [patch_features_temp_her2[x] for x in her2_indexes] 

**DATASET SPLIT** 

In [ ]:
train_her2_df = her2_df[her2_df['WSI No.'] < 101] 
val_her2_df = her2_df[(her2_df['WSI No.'] >= 101) & (her2_df['WSI No.'] < 111)] 
test_her2_df = her2_df[her2_df['WSI No.'] >= 111] 

train_her2_feats = patch_features_temp_her2[:len(train_her2_df)] 
val_her2_feats = patch_features_temp_her2[len(train_her2_df):len(train_her2_df) + len(val_her2_df)] 
test_her2_feats = patch_features_temp_her2[len(train_her2_df) + len(val_her2_df):] 

train_tnbc_df = tnbc_df[tnbc_df['WSI No.'] < 942] 
val_tnbc_df = tnbc_df[(tnbc_df['WSI No.'] >= 942) & (tnbc_df['WSI No.'] < 952)] 
test_tnbc_df = tnbc_df[tnbc_df['WSI No.'] >= 952] 

train_tnbc_feats = patch_features_temp_tnbc[:len(train_tnbc_df)] 
val_tnbc_feats = patch_features_temp_tnbc[len(train_tnbc_df):len(train_tnbc_df) + len(val_tnbc_df)] 
test_tnbc_feats = patch_features_temp_tnbc[len(train_tnbc_df) + len(val_tnbc_df):] 

train_df = pd.concat([train_her2_df, train_tnbc_df]) 
val_df = pd.concat([val_her2_df, val_tnbc_df]) 
test_df = pd.concat([test_her2_df, test_tnbc_df]) 

train_feats = train_her2_feats + train_tnbc_feats 
val_feats = val_her2_feats + val_tnbc_feats 
test_feats = test_her2_feats + test_tnbc_feats 

In [ ]:
train_wsi_nos = train_df['WSI No.'].to_list() 
train_ann_nos = train_df['Annotation Id'].to_list() 
train_labels_ = train_df['Label'].to_list() 

val_wsi_nos = val_df['WSI No.'].to_list() 
val_ann_nos = val_df['Annotation Id'].to_list() 
val_labels_ = val_df['Label'].to_list() 

test_wsi_nos = test_df['WSI No.'].to_list() 
test_ann_nos = test_df['Annotation Id'].to_list() 
test_labels_ = test_df['Label'].to_list() 

**AUTOENCODER FOR DIMENSIONALITY REDUCTION** 

In [ ]:
# Defining Autoencoder model

class Autoencoder(nn.Module):
    def __init__(self, input_size, encoding_dim):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
           nn.Linear(input_size, 16),
           nn.ReLU(),
           nn.Linear(16, encoding_dim),
           nn.ReLU()
        )
        self.decoder = nn.Sequential(
           nn.Linear(encoding_dim, 16),
           nn.ReLU(),
           nn.Linear(16, input_size),
           nn.Sigmoid()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(train_feats) 

In [ ]:
# Setting random seed for reproducibility
torch.manual_seed(42)

input_size = 1000  # Number of input features
encoding_dim = 224  # Desired number of output dimensions
model = Autoencoder(input_size, encoding_dim)
#model = model.to(device) 

# Loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr={LEARNING_RATE})

# Training the autoencoder
num_epochs = {NUMBER-OF-EPOCHS}
X_tensor = torch.FloatTensor(X_scaled) 
#X_tensor = X_tensor.to(device) 

for epoch in range(num_epochs):
    # Forward pass
    outputs = model(X_tensor)
    loss = criterion(outputs, X_tensor)

    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Loss for each epoch
    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

In [ ]:
# Combining feature maps 

combined_features = [] 
combined_features.extend(train_feats) 
combined_features.extend(val_feats) 
combined_features.extend(test_feats) 

scaler = StandardScaler()
X_scaled = scaler.fit_transform(combined_features) 
X_tensor = torch.FloatTensor(X_scaled) 
X_tensor = X_tensor.to(device) 

In [ ]:
# Encoding the data using the trained autoencoder 

encoded_feats = model.encoder(X_tensor).detach().cpu().numpy() 

train_encoded_feats = encoded_feats[:len(train_feats)] 
val_encoded_feats = encoded_feats[len(train_feats):len(train_feats) + len(val_feats)] 
test_encoded_feats = encoded_feats[len(train_feats) + len(val_feats):] 

**BAG FORMATION** 

In [ ]:
# Function to create and return bags (1 bag / annotation) for each dataset 

def create_bags(wsis, anns, feature_maps, labels_, min_patches, max_patches): 
    
    bags = [] 
    labels = [] 
    
    wsis_against_bags = []  # list to maintain wsi numbers against bags 
    
    count = 0 
    extra_bags = 0 
    
    discarded_anns = 0 
    total_anns = 0 
    
    total_wsis = 1 
    discarded_wsis = 0 
    prev_wsi_len = 0 
    
    i = 0 
    reserve = []
    
    # loop to traverse through each WSI 
    while (i < len(wsis)): 
        
        curr_wsi = wsis[i] 
        bag = [] 
        curr_ann = anns[i] 
        
        # initial length of bag before traversing through annotation 
        prev_len = len(bags) 
        
        # loop to traverse through each annotation 
        while (i < len(anns) and curr_ann == anns[i] and curr_wsi == wsis[i]): 
            curr_ann = anns[i] 
            bag.append(feature_maps[i]) 
            i += 1 
        
        # minimum number of patches in a bag to keep it 
        if len(bag) >= min_patches: 
            # maximum number of patches in a bag before splitting it 
            if len(bag) < max_patches: 
                bags.append(bag) 
                labels.append(labels_[i - 1]) 
                wsis_against_bags.append(curr_wsi) 
            
            else: 
                x = 0 
                while (x + max_patches) < len(bag): 
                    bags.append(bag[x : x + max_patches]) 
                    labels.append(labels_[i - 1]) 
                    wsis_against_bags.append(curr_wsi) 
                    x += max_patches 
        
        # len(bag) < min patches 
        else: 
            if i < len(wsis) and wsis[i] == curr_wsi: 
                reserve.extend(bag) 
        
        # new length of bag after traversing through annotation 
        new_len = len(bags) 
        
        # new wsi 
        if i < len(wsis) and wsis[i] != curr_wsi: 

            extra_bags_size = int(min_patches * (3 / 2)) 
            
            x = 0 
            # loop to off load reserve once done with a given wsi (min_patches x 2 instances in each extra bag) 
            while (x + extra_bags_size) <= len(reserve): 
                extra_bags += 1 
                bags.append(reserve[x : x + extra_bags_size]) 
                labels.append(labels_[i - 1]) 
                wsis_against_bags.append(curr_wsi) 
                x += extra_bags_size 
                
            # refresh reserve for next wsi 
            reserve = [] 
            
            # if wsi discarded 
            total_wsis += 1 
            if len(bags) == prev_wsi_len: 
                discarded_wsis += 1 
            prev_wsi_len = len(bags) 
        
        # if annotation discarded 
        if prev_len == new_len: 
            discarded_anns += 1 
        total_anns += 1 
    
    # calculating percentage of discarded wsis for a given min_patches value 
    perc_wsis_discarded = discarded_wsis / total_wsis 
    
    return bags, extra_bags, perc_wsis_discarded, wsis_against_bags, labels

In [ ]:
# creating bags 

train_bags, _, _, train_wsis, train_labels = create_bags(train_wsi_nos, train_ann_nos, train_encoded_feats, train_labels_, 50, 224) 
val_bags, _, _, val_wsis, val_labels = create_bags(val_wsi_nos, val_ann_nos, val_encoded_feats, val_labels_, 50, 224) 
test_bags, _, _, test_wsis, test_labels = create_bags(test_wsi_nos, test_ann_nos, test_encoded_feats, test_labels_, 50, 224) 

In [ ]:
print(len(train_bags), len(train_wsis), len(train_labels)) 
print(len(val_bags), len(val_wsis), len(val_labels)) 
print(len(test_bags), len(test_wsis), len(test_labels)) 

In [ ]:
all_bags = train_bags + val_bags + test_bags 
all_bag_labels = train_labels + val_labels + test_labels 
wsi_ids = train_wsis + val_wsis + test_wsis 

**CLASS DISTRIBUTION** 

In [ ]:
# class distribution of train, validation, and test datasets 

def plot_bag_labels(ax, title, labels):
    ax.bar(['HER2+', 'TNBC'], [labels.count(1), labels.count(0)])
    ax.set_title(title)
    ax.set_ylabel('Count')

fig, axs = plt.subplots(1, 3, figsize=(10, 4))

plot_bag_labels(axs[0], 'TRAIN BAG LABELS', train_labels)
plot_bag_labels(axs[1], 'VALIDATION BAG LABELS', val_labels)
plot_bag_labels(axs[2], 'TEST BAG LABELS', test_labels)

plt.tight_layout()
plt.show()

**CUSTOM DATASET CLASS** 

In [ ]:
# Custom dataset to load feature vectors & binary class labels 

class CustomDataset(Dataset):
    def __init__(self, bags, labels):
        self.bags = bags
        self.labels = labels

    def __len__(self):
        return len(self.bags)

    def __getitem__(self, idx):
        
        x = self.bags[idx]
        if len(x) > 224:
            feat_map = x[:224]
        else:
            feat_map = []
            feat_map.extend([np.array([0.0] * 224)] * int((224 - len(x)) / 2)) # black padding above
            feat_map.extend(x) # feature map of patches in between
            feat_map.extend([np.array([0.0] * 224)] * int(((224 - len(x)) / 2) + 1)) # black padding below
            feat_map = feat_map[:224]
        
        x = np.array(feat_map) 
        
        x = torch.tensor(x).float()
        x = nn.functional.normalize(x, dim=0, p=2)  # p=2 for L2 norm, dim=0 for cols
        bag = x
        label = self.labels[idx]

        return torch.stack([bag]), label

**FINAL MODEL** 

In [ ]:
# defining final pipeline to process prepared bags 

class Pipeline(nn.Module):
    def __init__(self,num_classes):
        #define necessary layers
        super().__init__()
        self.num_classes = num_classes
          
        self.base = models.{'MODEL_NAME'}(weights=True)
        
        # Unfreeze model weights
        for param in self.base.parameters():
            param.requires_grad = True
        
        self.flatten = nn.Flatten()
        
        self.head = nn.Sequential(
            nn.Linear(1000, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        ) 
    
    def forward(self,X):
        X = self.base(X)
        X = self.flatten(X)
        X = self.head(X)
        return X, F.sigmoid(X)

In [ ]:
!export CUDA_LAUNCH_BLOCKING=1

**5 FOLD CROSS VALIDATION** 

In [ ]:
num_epochs = {NUMBER-OF-EPOCHS}

In [ ]:
# Set number of folds and random seed
num_folds = {NUM_FOLDS}
batch_size = {BATCH_SIZE}
num_of_workers = {NUMBER-OF-WORKERS}
np.random.seed(42)  # Ensures reproducibility for shuffling

# Shuffle and split WSIs into groups with exactly 2 WSIs per fold
unique_wsis = list(set(wsi_ids))
np.random.shuffle(unique_wsis)  # Shuffle to randomize groupings

# Divide unique WSIs into folds, each fold containing 2 WSIs
wsi_groups = [unique_wsis[i:i + 25] for i in range(0, len(unique_wsis), 25)]

metrics = {
    fold + 1: {
        'train_loss': [], 'train_accuracy': [], 'train_roc_auc': [],
        'valid_loss': [], 'valid_accuracy': [], 'valid_roc_auc': [],
        'wsi_nos': [], 'ann_nos': [], 'y_pred': [], 'y_true': []
    } for fold in range(num_folds)
}

# Run custom cross-validation
for fold in range(num_folds):
    test_wsis = wsi_groups[fold]  # Select two WSIs for this fold's test set
    train_wsis = [wsi for group in wsi_groups if group != test_wsis for wsi in group]
    
    # Get indices for training and testing based on selected WSIs
    train_idx = [i for i, wsi in enumerate(wsi_ids) if wsi in train_wsis]
    test_idx = [i for i, wsi in enumerate(wsi_ids) if wsi in test_wsis]
    
    # Splitting bags and labels for each fold without converting to numpy array
    train_bags = [all_bags[i] for i in train_idx]
    test_bags = [all_bags[i] for i in test_idx]
    train_labels = [all_bag_labels[i] for i in train_idx]
    test_labels = [all_bag_labels[i] for i in test_idx]

    metrics[fold + 1]['wsi_nos'] = [wsi_ids[i] for i in test_idx] 

    # Creating datasets and loaders for each fold
    train_dataset = CustomDataset(train_bags, train_labels)
    val_dataset = CustomDataset(test_bags, test_labels)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_of_workers)
    valid_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=num_of_workers)

    # Output the fold details
    print(f"\nFold {fold + 1}")
    print(f"Train WSI IDs: {set(np.array(wsi_ids)[train_idx])}") 
    print(f"Train size: {len(train_loader.dataset)}, Test size: {len(valid_loader.dataset)}")

    # Initialize empty lists to store loss and accuracy for training and validation
    train_losses = []
    valid_losses = []
    train_accuracies = []
    valid_accuracies = []
    roc_values_train = []
    roc_values_val = []
    
    pos_weight = torch.tensor([POSITIVE-WEIGHT])
    pos_weight = pos_weight.to(device)
    loss_fn = nn.BCEWithLogitsLoss(pos_weight = pos_weight)
    model = Pipeline(1)
    
    # modifying first layer to input color channel. Similarly can be done for other models
    # for efficientnetb0
    model.base.features[0][0] = nn.Conv2d(1, 32,kernel_size= (3,3), stride = 2, padding= 1, bias=False)
    
    # for resnet50
    #model.base.conv1=nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    
    # for VIT
    #model.base.conv_proj = nn.Conv2d(1, 768, kernel_size=(32, 32), stride=(32, 32))
    
    model = model.to(device) 
    optimizer = optim.Adam(params=model.parameters(), lr={LEARNING_RATE}) 
    
    for epoch in range(num_epochs):
        model.train()
        y_true_train = []
        y_scores_train = []
        train_loss = 0
        correct = 0
        total = 0
        for images, labels in tqdm(train_loader):
            images = images.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            outputs_without_sigmoid, outputs_with_sigmoid = model(images)
            loss = loss_fn(outputs_without_sigmoid.squeeze(-1), labels.float())
            loss.backward()
            optimizer.step()
            train_loss += loss.item()  # L+=l.item()
            predicted = (outputs_with_sigmoid >= 0.5).long().squeeze(-1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            y_true_train.extend(labels.cpu().detach().numpy())
            y_scores_train.extend(outputs_with_sigmoid.cpu().detach().numpy())

        train_losses.append(train_loss / len(train_loader))
        train_accuracies.append(100 * correct / total)

        # roc auc logic
        fpr_train, tpr_train, _ = roc_curve(y_true_train, y_scores_train)
        roc_auc_train = auc(fpr_train, tpr_train)
        roc_values_train.append(roc_auc_train)

        # Plot ROC curve for the training set

        # Validate your model after each epoch if needed
        model.eval()
        valid_loss = 0
        correct = 0
        total = 0
        y_true_val = []
        y_scores_val = []

        with torch.no_grad():
            for images, labels in tqdm(valid_loader):
                # Move data to GPU if available
                images = images.to(device)
                labels = labels.to(device)
                outputs_without_sigmoid, outputs_with_sigmoid = model(images)
                loss = loss_fn(outputs_without_sigmoid.squeeze(-1), labels.float())
                valid_loss += loss.item()  # L+=l.item()
                predicted = (outputs_with_sigmoid >= 0.5).long().squeeze(-1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

                y_true_val.extend(labels.cpu().detach().numpy())
                y_scores_val.extend(outputs_with_sigmoid.cpu().detach().numpy())

        valid_losses.append(valid_loss / len(valid_loader))
        valid_accuracies.append(100 * correct / total)
        # roc auc logic
        fpr_val, tpr_val, _ = roc_curve(y_true_val, y_scores_val)
        roc_auc_val = auc(fpr_val, tpr_val)
        roc_values_val.append(roc_auc_val)

        # Classification report
        print(f'Epoch {epoch + 1}, Train Accuracy: {train_accuracies[-1]:.2f}%, Train Loss: {train_losses[-1]:.2f}%, Train ROC-AUC: {roc_auc_train:.2f}%, Val Accuracy: {valid_accuracies[-1]:.2f}%, Val Loss: {valid_losses[-1]:.2f}%, Val ROC-AUC: {roc_auc_val:.2f}%')

        y_true_val = np.array(y_true_val)
        y_pred_val = (np.array(y_scores_val) >= 0.5).astype(int)
        print("Validation Classification Report:")
        print(classification_report(y_true_val, y_pred_val,zero_division=1))
        
         # Append metrics for current epoch and fold
        metrics[fold + 1]['train_loss'].append(train_losses[-1])
        metrics[fold + 1]['train_accuracy'].append(train_accuracies[-1])
        metrics[fold + 1]['train_roc_auc'].append(roc_values_train[-1])
        metrics[fold + 1]['valid_loss'].append(valid_losses[-1])
        metrics[fold + 1]['valid_accuracy'].append(valid_accuracies[-1])
        metrics[fold + 1]['valid_roc_auc'].append(roc_values_val[-1])
    metrics[fold + 1]['y_pred']=y_pred_val
    metrics[fold + 1]['y_true']=y_true_val

In [ ]:
# Plotting function for losses and accuracies by fold 

def plot_metrics(metrics, num_folds):
    metric_names = ['train_loss', 'valid_loss', 'train_accuracy', 'valid_accuracy', 'train_roc_auc', 'valid_roc_auc']
    for metric_name in metric_names:
        plt.figure(figsize=(10, 5))
        for fold in range(1, num_folds + 1):
            fold_metric = metrics[fold][metric_name]
            plt.plot(range(1, num_epochs + 1), fold_metric, label=f'Fold {fold}')
        plt.title(f'{metric_name.replace("_", " ").title()} over Epochs')
        plt.xlabel('Epoch')
        plt.ylabel(metric_name.replace('_', ' ').title())
        plt.legend()
        plt.show()

# Call the plotting function
plot_metrics(metrics, num_folds) 

**AGGREGATING BAG LEVEL RESULTS TO WSI LEVEL** 

In [ ]:
# Dictionary to store final WSI-level prediction counts across all folds
wsi_pred_counts = defaultdict(lambda: {0: 0, 1: 0})

# Loop over each fold and accumulate predictions
for fold in range(1, num_folds + 1):
    # Retrieve WSI IDs and predictions
    wsi_ids = metrics[fold]['wsi_nos']
    predicted_labels = metrics[fold]['y_pred']

    # Count 0s and 1s per WSI
    for wsi, pred in zip(wsi_ids, predicted_labels):
        pred = int(pred)  # Ensure hashable
        wsi_pred_counts[wsi][pred] += 1

wsi_pred_counts = dict(wsi_pred_counts) 
#wsi_pred_counts 

In [ ]:
# Lists to store WSI-level ground truths and predicted labels 

wsi_true_labels = []
wsi_pred_labels = []

# Loop over each WSI and compute majority vote
for item in wsi_pred_counts.items(): 

    true_label = df[df['WSI No.'] == item[0]]['Label'].to_list()[0] 
    wsi_true_labels.append(true_label)  # ground truth from external dict 
    majority_vote = 1 if item[1][1] >= item[1][0] else 0  # tie → label 1
    wsi_pred_labels.append(majority_vote) 

In [ ]:
# Classification report for test set 

print("Test WSI Level Classification Report:\n")
print(classification_report(wsi_true_labels, wsi_pred_labels)) 

In [ ]:
# calculating CI 

accuracies = []
for i in range(1000):
    y_true_sample, y_pred_sample = resample(wsi_true_labels, wsi_pred_labels)
    accuracies.append(accuracy_score(y_true_sample, y_pred_sample))

# Compute confidence interval
alpha = 0.95
lower = np.percentile(accuracies, (1 - alpha) / 2 * 100)
upper = np.percentile(accuracies, (1 + alpha) / 2 * 100)

print(f"Accuracy: {np.mean(accuracies):.3f} with 95% CI: [{lower:.3f}, {upper:.3f}]") 

In [ ]:
# Generate confusion matrix
conf_matrix = confusion_matrix(wsi_true_labels, wsi_pred_labels)

# Plot confusion matrix
plt.figure(figsize=(5, 5))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap="Blues", cbar=False)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('WSI Level Confusion Matrix')
plt.show()